# This notebook provides the commands needed to read in an NWB file with raw data taken from a trodes .rec file and to copy the relevant metadata into a DataJoint Schema designed to mimic NWB structure

# The notebook then filters the LFP data based on 

## The following steps must be completed before running this notebook:
### 1) Set up access to a DataJoint database. See https://tutorials.datajoint.io/
### 2) Install pynwb. 
    See https://pynwb.readthedocs.io/en/stable/getting_started.html#installation
### 3) Install the ndx-fl-novela package. 
     conda install -c novelakrk ndx-fl-novela
### 4) Download the example franklab NWB file from DropBox. 
    https://www.dropbox.com/to_be_updated

#### Make sure we're in the franklabnwb/franklabnwb directory. 
#### Eventually this will be unnecessary because the package will properly installed

In [2]:
cd ../schema

/Users/loren/Src/NWB/nwb_datajoint/schema


#### Load all of the relevant modules

In [3]:
%env DJ_SUPPORT_FILEPATH_MANAGEMENT=TRUE
%load_ext autoreload
%autoreload 2


import pynwb
import os

#DataJoint and DataJoint schema
import datajoint as dj

dj.config["enable_python_native_blobs"] = True
dj.config['stores'] = {
  'local': dict(  # 'regular' external storage for this pipeline
                protocol='file',
                location='/data/nwb_builder_test_data/dj/',
                stage='/data/nwb_builder_test_data/dj/'),
  'local-filtered': dict( # 'local_filtered' storage filtered data in hdf5 files
                protocol='file',
                location='/data/nwb_builder_test_data/dj/filtered/',
                stage='/data/nwb_builder_test_data/dj/filtered/')
}

import common_session
import common_ephys
import common_device
import common_lab
import common_task
import common_behav
import common_interval
import common_region
import common_filter
import nwb_dj
import ndx_franklab_novela.probe

#franklabnwb functions
#from franklabnwb import fl_ns_path
#import franklabnwb.nspike_helpers as ns
#import franklabnwb.query_helpers as qu
#import franklabnwb.fl_extension as fle
#import franklabnwb.fl_extension as flh

env: DJ_SUPPORT_FILEPATH_MANAGEMENT=TRUE
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Please enter DataJoint username:  root
Please enter DataJoint password:  ········


Connecting root@localhost:3306


20:20:45 [I] klustakwik KlustaKwik2 version 0.2.6


### Set up a datajoint file store to save NWB and filtered data files. Change directories to match your local setup

#### Next we specify the NWB file name and populate the rest of the schema
#### Replace the code below with the text for the path to the NWB file

In [4]:
nwb_raw_file_name = '/data/nwb_builder_test_data/beans20190718.nwb'

### temporary temporary file; FIX when NWB file handing sorted out

In [8]:
#common_session.Nwbfile().external['local'].delete(delete_external_files=True)

0it [00:00, ?it/s]


[]

In [5]:
#common_session.Nwbfile.drop()

In [6]:
nwb_file_name = '/data/nwb_builder_test_data/beans20190718_pp.nwb'

In [7]:
nwb_dj.NWBPopulate([nwb_raw_file_name]) 

/Users/loren/opt/anaconda3/envs/nwbdj/lib/python3.7/site-packages/pynwb/file.py:404: DeprecationWarning: replaced by NWBFile.processing
  warn("replaced by NWBFile.processing", DeprecationWarning)
/Users/loren/opt/anaconda3/envs/nwbdj/lib/python3.7/site-packages/pynwb/base.py:75: PendingDeprecationWarning: get_data_interface will be replaced by get
  warn(PendingDeprecationWarning('get_data_interface will be replaced by get'))


Importing raw data: Estimated sampling rate:	20000.0 Hz
                    Number of valid intervals:	5
Processing raw position data. Estimated sampling rate: 30.0 Hz


/Users/loren/opt/anaconda3/envs/nwbdj/lib/python3.7/site-packages/pynwb/file.py:404: DeprecationWarning: replaced by NWBFile.processing
  warn("replaced by NWBFile.processing", DeprecationWarning)
/Users/loren/opt/anaconda3/envs/nwbdj/lib/python3.7/site-packages/pynwb/base.py:75: PendingDeprecationWarning: get_data_interface will be replaced by get
  warn(PendingDeprecationWarning('get_data_interface will be replaced by get'))


#### Create the standard LFP Filters. 

In [8]:
common_filter.create_standard_filters()

### Select all electrodes for LFP

In [9]:
electrode_ids = common_ephys.ElectrodeConfig.Electrode.fetch('electrode_id')
common_ephys.LFPElectrode().set_lfp_elect(nwb_file_name, electrode_ids.tolist())


About to delete:
Nothing to delete


### Or select one electrode for LFP


In [9]:
common_ephys.LFPElectrode().set_lfp_elect(nwb_file_name, [2])


About to delete:
Nothing to delete


### populate the LFP tabke

In [10]:
common_ephys.LFP().populate()

Output array should have shape (1091950, 0) and dtype <f8
Output array should have shape (212812, 0) and dtype <f8
Output array should have shape (2557744, 0) and dtype <f8
Output array should have shape (1356939, 0) and dtype <f8
Output array should have shape (1680945, 0) and dtype <f8
Checking output array shape is disabled, make sure portion of output array has shape (1091950, 0)
Checking output array shape is disabled, make sure portion of output array has shape (212812, 0)
Checking output array shape is disabled, make sure portion of output array has shape (2557744, 0)
Checking output array shape is disabled, make sure portion of output array has shape (1356939, 0)
Checking output array shape is disabled, make sure portion of output array has shape (1680945, 0)


/Users/loren/opt/anaconda3/envs/nwbdj/lib/python3.7/site-packages/datajoint/connection.py:218: UserWarning: MySQL server has gone away. Reconnecting to the server.
  warnings.warn("MySQL server has gone away. Reconnecting to the server.")


LostConnectionError: Connection was lost during a transaction.

In [12]:
common_ephys.LFP()

nwb_file_name,electrode_id the unique number for this electrode,linked_file_name the name of each linked file,interval_name descriptive name of this interval list,filter_name descriptive name of this filter,filter_sampling_rate sampling rate for this filter,nwb_object_id the NWB object ID for loading this object from the linked file,"sampling_rate the sampling rate, in HZ"
/data/nwb_builder_test_data/beans20190718_pp.nwb,1,/data/nwb_builder_test_data/beans20190718_pp00000002.nwb,raw data valid times,LFP 0-400 Hz,20000,54b3a111-1fbc-4635-bd42-b5466dbf990d,20000.0


In [41]:
common_session.LinkedNwbfile()

nwb_file_name,linked_file_name the name of each linked file
/data/nwb_builder_test_data/beans20190718_pp.nwb,/data/nwb_builder_test_data/beans20190718_pp00000000.nwb
